## Loading data

In [0]:
from azure.storage.filedatalake import DataLakeServiceClient

# Replace with your details
storage_account_name = 
storage_account_key = 

# Connect to ADLS
service_client = DataLakeServiceClient(
    account_url=f"https://{storage_account_name}.dfs.core.windows.net",
    credential=storage_account_key,
    api_version="2023-11-03"  # Use the correct supported API version
)

# List Containers
containers = service_client.list_file_systems()
for container in containers:
    print(container.name)


amzecomdata
globalmartmarketingdata
inventorydata
optimalchannel


In [0]:
# Replace with your details
container_name = 

In [0]:
import os
from azure.storage.blob import BlobServiceClient
import pandas as pd
import io

def read_csv_from_blob(storage_account_name, container_name, file_name, storage_account_key=None):
    """
    Read a CSV file from Azure Blob Storage using Python and return a Pandas DataFrame.

    :param storage_account_name: Azure storage account name.
    :param container_name: Blob container name.
    :param file_name: Name of the file in the container.
    :param storage_account_key: Storage account access key.
    :return: Pandas DataFrame.
    """
    
    if not storage_account_key:
        # Try to get the key from environment variables if not provided
        storage_account_key = os.environ.get('AZURE_STORAGE_KEY')
        
    if not storage_account_key:
        raise ValueError("Storage account key must be provided either as a parameter or as an environment variable 'AZURE_STORAGE_KEY'")
    
    try:
        # Create a connection string
        connection_string = f"DefaultEndpointsProtocol=https;AccountName={storage_account_name};AccountKey={storage_account_key};EndpointSuffix=core.windows.net"
        
        # Create the BlobServiceClient
        blob_service_client = BlobServiceClient.from_connection_string(connection_string)
        
        # Get the container client
        container_client = blob_service_client.get_container_client(container_name)
        
        # Get the blob client
        blob_client = container_client.get_blob_client(file_name)
        
        # Download the blob content
        download_stream = blob_client.download_blob()
        
        # Convert the content to a DataFrame
        content = download_stream.readall()
        df = pd.read_csv(io.BytesIO(content))
        
        return df
    
    except Exception as e:
        print(f"Error reading CSV file: {e}")
        return None

In [0]:
online_marketing = read_csv_from_blob(storage_account_name=storage_account_name,
                                      container_name=container_name, 
                                      file_name="Online_Marketing_Data.csv",
                                      storage_account_key=storage_account_key)

offline_marketing = read_csv_from_blob(storage_account_name=storage_account_name,
                                      container_name=container_name, 
                                      file_name="Offline_Marketing_Data.csv",
                                      storage_account_key=storage_account_key)
sales = read_csv_from_blob(storage_account_name=storage_account_name,
                                      container_name=container_name, 
                                      file_name="Sales_Data.csv",
                                      storage_account_key=storage_account_key)
price = read_csv_from_blob(storage_account_name=storage_account_name,
                                      container_name=container_name, 
                                      file_name="Pricing_Data.csv",
                                      storage_account_key=storage_account_key)

## Media Data Processing
* Online Marketing data 
* Offline Marketing data

In [0]:
online_marketing.head()

,date,channel,spend,impressions,clicks
0,2023-01-01,Facebook,1152.82,20117,613
1,2023-01-01,Instagram,707.48,9760,196
2,2023-01-01,YouTube,703.13,11148,236
3,2023-01-01,Google Ads,810.68,12761,553
4,2023-01-01,Influencer Marketing,1122.12,22415,554


In [0]:
# Convert the 'date' column in the online marketing dataset to datetime format
# This ensures consistent date formatting for further processing
online_marketing['date'] = pd.to_datetime(online_marketing['date'])
online_marketing['week'] = online_marketing['date'] - pd.to_timedelta(online_marketing['date'].dt.weekday, unit='d')

#drop date column as we now have week column
media_weekly = online_marketing.drop(['date'], axis=1)
media_weekly = media_weekly.groupby(['week', 'channel']).sum().reset_index() # .agg({})

offline_marketing['week'] = pd.to_datetime(offline_marketing['week'])  # Convert 'week' to datetime format
media = pd.merge(
    media_weekly,
    offline_marketing,
    on=['week', 'channel'],
    how='outer',  # Include all rows from both datasets (outer join)
    suffixes=('_online', '_offline')  # Differentiate overlapping column names
) 

# Calculate CTR (Click-Through Rate)
media['CTR (%)'] = (media['clicks'] / media['impressions']) * 100

# Handle rows where impressions might be zero to avoid division by zero
media['CTR (%)'] = media['CTR (%)'].fillna(0)

media.head()


,week,channel,spend_online,impressions,clicks,spend_offline,CTR (%)
0,2022-12-26,Facebook,1152.82,20117.0,613.0,NaN,3.047174
1,2022-12-26,Google Ads,810.68,12761.0,553.0,NaN,4.333516
2,2022-12-26,Influencer Marketing,1122.12,22415.0,554.0,NaN,2.471559
3,2022-12-26,Instagram,707.48,9760.0,196.0,NaN,2.008197
4,2022-12-26,YouTube,703.13,11148.0,236.0,NaN,2.116972


## Sales Data Pre Processing
* Sales data 
* Pricing data

In [0]:
sales.head()

,date,city,state,store_id,product_id,sales_quantity
0,2023-01-01,Mumbai,Maharashtra,MUM_01,P001,112
1,2023-01-01,Mumbai,Maharashtra,MUM_01,P002,100
2,2023-01-01,Mumbai,Maharashtra,MUM_01,P003,126
3,2023-01-01,Mumbai,Maharashtra,MUM_01,P004,108
4,2023-01-01,Mumbai,Maharashtra,MUM_01,P005,129


In [0]:
price.head()

,date,city,state,store_id,product_id,base_price,promotional_discount,promotion_type,final_price
0,01-01-2023,Mumbai,Maharashtra,MUM_01,P001,134.35,6.72,Percentage Discount,127.63
1,01-01-2023,Mumbai,Maharashtra,MUM_02,P001,135.09,13.51,Percentage Discount,121.58
2,01-01-2023,Mumbai,Maharashtra,MUM_03,P001,136.33,0.00,None,136.33
3,01-01-2023,Mumbai,Maharashtra,MUM_04,P001,140.32,0.00,None,140.32
4,01-01-2023,Mumbai,Maharashtra,MUM_05,P001,142.32,14.23,Buy One Get One Free,128.09


In [0]:
sales['date'] = pd.to_datetime(sales['date'], errors='coerce')
price['date'] = pd.to_datetime(price['date'], errors='coerce')

In [0]:
# Merge Sales and Pricing data
# - Combine sales and pricing datasets based on common columns:
#   'date', 'city', 'state', 'store_id', and 'product_id'.
# - Use an inner join to retain only the rows where there are matches in both datasets.
merged_sales_pricing = pd.merge(
    sales,
    price,
    on=['date', 'city', 'state', 'store_id', 'product_id'],
    how='inner'
)

merged_sales_pricing.head()

,date,city,state,store_id,product_id,sales_quantity,base_price,promotional_discount,promotion_type,final_price
0,2023-01-01,Mumbai,Maharashtra,MUM_01,P001,112,134.35,6.72,Percentage Discount,127.63
1,2023-01-01,Mumbai,Maharashtra,MUM_01,P002,100,114.54,11.45,Percentage Discount,103.09
2,2023-01-01,Mumbai,Maharashtra,MUM_01,P003,126,134.44,6.72,None,127.72
3,2023-01-01,Mumbai,Maharashtra,MUM_01,P004,108,107.04,10.70,Buy One Get One Free,96.34
4,2023-01-01,Mumbai,Maharashtra,MUM_01,P005,129,133.43,6.67,None,126.76


In [0]:
# Calculate Sales Amount
# - Create a new column 'sales_amount' by multiplying the quantity of products sold
#   ('sales_quantity') with the 'final_price' of the product.
merged_sales_pricing['sales_amount'] = (
    merged_sales_pricing['sales_quantity'] * merged_sales_pricing['final_price'])

In [0]:
# Convert 'date' to datetime and create a 'week' column
# - Convert the 'date' column to datetime format for consistency in processing.
# - Derive the 'week' column to represent weekly aggregation.
#   - Subtract the current weekday to align the date to the start of the week.
#   - Add an offset to ensure the week starts on Sunday (weekday=6).
merged_sales_pricing['date'] = pd.to_datetime(merged_sales_pricing['date'])
merged_sales_pricing['week'] = (
    merged_sales_pricing['date']
    - pd.to_timedelta(merged_sales_pricing['date'].dt.weekday, unit='D')
    + pd.offsets.Week(weekday=6)
)

In [0]:
merged_sales_pricing.head()

,date,city,state,store_id,product_id,sales_quantity,base_price,promotional_discount,promotion_type,final_price,sales_amount,week
0,2023-01-01,Mumbai,Maharashtra,MUM_01,P001,112,134.35,6.72,Percentage Discount,127.63,14294.56,2023-01-01
1,2023-01-01,Mumbai,Maharashtra,MUM_01,P002,100,114.54,11.45,Percentage Discount,103.09,10309.00,2023-01-01
2,2023-01-01,Mumbai,Maharashtra,MUM_01,P003,126,134.44,6.72,None,127.72,16092.72,2023-01-01
3,2023-01-01,Mumbai,Maharashtra,MUM_01,P004,108,107.04,10.70,Buy One Get One Free,96.34,10404.72,2023-01-01
4,2023-01-01,Mumbai,Maharashtra,MUM_01,P005,129,133.43,6.67,None,126.76,16352.04,2023-01-01


In [0]:
# Convert 'date' to datetime and create a 'week' column
# - Convert the 'date' column to datetime format for consistency in processing.
# - Derive the 'week' column to represent weekly aggregation.
#   - Subtract the current weekday to align the date to the start of the week.
#   - Add an offset to ensure the week starts on Sunday (weekday=6).
merged_sales_pricing['date'] = pd.to_datetime(merged_sales_pricing['date'])
merged_sales_pricing['week'] = (
    merged_sales_pricing['date']
    - pd.to_timedelta(merged_sales_pricing['date'].dt.weekday, unit='D')
    + pd.offsets.Week(weekday=6)
)

In [0]:
# Group by week and aggregate other columns logically
sales_amt = merged_sales_pricing.groupby('week').agg({
    'sales_amount': 'sum',  # Total sales amount
    'base_price': 'mean',  # Average base price
    'final_price': 'mean',  # Average final price
    'promotion_type': lambda x: x.mode().iloc[0] if not x.mode().empty else None  # Mode of promotion type 
}).reset_index()

In [0]:
sales_amt.head()

,week,sales_amount,base_price,final_price,promotion_type
0,2023-01-01,13516527.77,101.830513,94.148539,None
1,2023-01-08,95262872.49,102.025962,94.308786,None
2,2023-01-15,54399888.16,102.019562,94.481334,None


## Final data

In [0]:
# Merge the two dataframes on the `week` column
merged_df = pd.merge(sales_amt, media, on='week', how='left')

In [0]:
merged_df.head()

,week,sales_amount,base_price,final_price,promotion_type,channel,spend_online,impressions,clicks,spend_offline,CTR (%)
0,2023-01-01,13516527.77,101.830513,94.148539,None,TV,NaN,NaN,NaN,11311.42,0.0
1,2023-01-01,13516527.77,101.830513,94.148539,None,Radio,NaN,NaN,NaN,6723.33,0.0
2,2023-01-01,13516527.77,101.830513,94.148539,None,Print,NaN,NaN,NaN,6214.43,0.0
3,2023-01-01,13516527.77,101.830513,94.148539,None,OOH,NaN,NaN,NaN,11230.47,0.0
4,2023-01-08,95262872.49,102.025962,94.308786,None,TV,NaN,NaN,NaN,14004.01,0.0
